In [1]:
import pandas as pd
import numpy as np
import joblib

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier, IsolationForest
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier


In [2]:
import pandas as pd

df = pd.read_excel(r"C:\Users\ASUS\Desktop\python_p\data\final_dataset.xlsx")

df.head()


,patient_id,policy_number,claim_id,claim_date,service_date,policy_expiration_date,claim_amount,patient_age,patient_gender,patient_city,...,diagnosis,fraud_type,claim_amount_per_day,claim_amount_per_procedure,high_risk_provider,repeated_diagnosis_flag,unusual_distance_flag,claim_frequency_score,patient_risk_score,provider_risk_score
0,67349080.0,XAI956194550,18874.0,2021-06-16,2021-05-01,2024-11-20,462337.48,28.0,Female,Houston,...,A09,Non-Fraud,9.836968e+03,2.311687e+05,0.0,0.0,0.0,0.0,0.0,0.470588
1,85973291.0,XAI215993963,8762.0,2025-01-19,2025-01-17,2029-04-07,1883481.3,40.0,Other,Washington,...,J02.9,Potential Claim Fraud,1.883481e+06,9.417407e+05,0.0,0.0,0.0,0.0,0.3,0.176471
2,62454860.0,XAI146653263,1920.0,2023-05-05,2023-03-24,2024-09-04,1500658.89,90.0,Female,Seattle,...,I25.10,Non-Fraud,1.500659e+06,1.500659e+06,0.0,0.0,0.0,0.0,0.3,0.058824
3,58632240.0,XAI216596381,1316.0,2021-11-07,2021-11-02,2025-12-24,572237.61,49.0,Other,Fort Worth,...,I10,Potential Claim Fraud,5.722376e+05,1.430594e+05,0.0,0.0,0.0,0.0,0.0,0.058824
4,96347311.0,XAI056900359,17039.0,2024-03-25,2024-02-18,2028-07-30,2088293.84,24.0,Female,New York,...,H25.9,Non-Fraud,2.088294e+06,4.176588e+05,1.0,0.0,0.0,0.0,0.3,0.441176


In [3]:
print("Shape of dataset:", df.shape)
print("\nColumn names:\n", df.columns)
print("\nFraud Distribution:\n", df['is_fraudulent'].value_counts())
print("\nFraud Percentage:\n", df['is_fraudulent'].value_counts(normalize=True) * 100)


Shape of dataset: (3668, 43)

Column names:
 Index(['patient_id', 'policy_number', 'claim_id', 'claim_date', 'service_date',
       'policy_expiration_date', 'claim_amount', 'patient_age',
       'patient_gender', 'patient_city', 'patient_state', 'hospital_id',
       'provider_type', 'provider_specialty', 'provider_city',
       'provider_state', 'diagnosis_code', 'procedure_code',
       'number_of_procedures', 'admission_type', 'discharge_type',
       'length_of_stay_days', 'service_type', 'deductible_amount',
       'copay_amount', 'number_of_previous_claims_patient',
       'number_of_previous_claims_provider', 'provider_patient_distance_miles',
       'claim_submitted_late', 'is_fraudulent', 'date of encounter',
       'date of discharge', 'amount billed', 'diagnosis', 'fraud_type',
       'claim_amount_per_day', 'claim_amount_per_procedure',
       'high_risk_provider', 'repeated_diagnosis_flag',
       'unusual_distance_flag', 'claim_frequency_score', 'patient_risk_score',
   

In [4]:
import pandas as pd
import numpy as np

# Remove duplicates
df.drop_duplicates(inplace=True)
print("After removing duplicates:", df.shape)

# Handle missing values
df.fillna(method='ffill', inplace=True)

# Drop ID-like columns if present
id_cols = ['patient_id', 'claim_id', 'policy_number']
df.drop(columns=[col for col in id_cols if col in df.columns], inplace=True)

# Separate features and target
target_col = 'is_fraudulent'
X = df.drop(columns=[target_col])
y = df[target_col]

print("Features shape:", X.shape)
print("Target distribution:\n", y.value_counts())


After removing duplicates: (3666, 43)
Features shape: (3666, 39)
Target distribution:
 is_fraudulent
0.0    2720
1.0     946
Name: count, dtype: int64


C:\Users\ASUS\AppData\Local\Temp\ipykernel_28804\3975887964.py:9: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', inplace=True)


In [5]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Identify column types
categorical_cols = X.select_dtypes(include='object').columns
numeric_cols = X.select_dtypes(exclude='object').columns

# Preprocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
    ]
)

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("Train size:", X_train.shape)
print("Test size:", X_test.shape)


Train size: (2932, 39)
Test size: (734, 39)


In [6]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

categorical_cols = X.select_dtypes(include='object').columns
numeric_cols = X.select_dtypes(exclude='object').columns

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
    ]
)


In [7]:
for col in X.columns:
    types = X[col].apply(type).value_counts()
    if len(types) > 1:
        print(f"Column '{col}' has mixed types:\n{types}\n")


Column 'claim_amount' has mixed types:
claim_amount
<class 'float'>    3639
<class 'int'>        26
<class 'str'>         1
Name: count, dtype: int64



In [8]:
# Fill missing values first
X = X.fillna("Unknown")

# Convert ALL object columns to string
for col in X.select_dtypes(include=['object']).columns:
    X[col] = X[col].astype(str)


In [9]:
for col in X.select_dtypes(exclude=['object']).columns:
    X[col] = pd.to_numeric(X[col], errors='coerce')

# Fill any new NaNs created
X.fillna(X.median(numeric_only=True), inplace=True)


In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


In [11]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

categorical_cols = X.select_dtypes(include='object').columns
numeric_cols = X.select_dtypes(exclude='object').columns

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
    ]
)


In [12]:
df = df.copy()

# 1. Claim per day (fraudsters inflate bills for short stays)
if 'Claim_Amount' in df.columns and 'Length_of_Stay' in df.columns:
    df['Claim_Per_Day'] = df['Claim_Amount'] / (df['Length_of_Stay'] + 1)

# 2. Claim per procedure
if 'Claim_Amount' in df.columns and 'Number_of_Procedures' in df.columns:
    df['Claim_Per_Procedure'] = df['Claim_Amount'] / (df['Number_of_Procedures'] + 1)

# 3. Total hospital cost
cost_cols = ['Doctor_Fee','Medicine_Cost','Room_Charges','ICU_Charges']
existing = [c for c in cost_cols if c in df.columns]
if existing:
    df['Total_Cost'] = df[existing].sum(axis=1)

# 4. High previous claims indicator
if 'Previous_Claims' in df.columns:
    df['Frequent_Claimer'] = (df['Previous_Claims'] > df['Previous_Claims'].median()).astype(int)

# 5. Age risk group
if 'Age' in df.columns:
    df['Age_Group'] = pd.cut(df['Age'], bins=[0,25,60,100], labels=['Young','Adult','Senior'])


In [13]:
X = df.drop(columns=['is_fraudulent'])
y = df['is_fraudulent']


In [14]:
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import SMOTE

smote = SMOTE(k_neighbors=2, random_state=42)


In [15]:
from sklearn.ensemble import RandomForestClassifier

rf_model = ImbPipeline(steps=[
    ('preprocess', preprocessor),
    ('smote', smote),
    ('model', RandomForestClassifier(
        n_estimators=600,
        max_depth=None,
        min_samples_split=5,
        min_samples_leaf=2,
        max_features='sqrt',
        class_weight='balanced_subsample',
        random_state=42
    ))
])

rf_model.fit(X_train, y_train)
print("RF trained")


RF trained


In [16]:
from sklearn.linear_model import LogisticRegression

lr_model = ImbPipeline(steps=[
    ('preprocess', preprocessor),
    ('smote', smote),
    ('model', LogisticRegression(max_iter=3000, C=0.5))
])

lr_model.fit(X_train, y_train)


,steps,"[('preprocess', ...), ('smote', ...), ...]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('num', ...), ('cat', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [17]:
from sklearn.tree import DecisionTreeClassifier

dt_model = ImbPipeline(steps=[
    ('preprocess', preprocessor),
    ('smote', smote),
    ('model', DecisionTreeClassifier(max_depth=12, min_samples_split=10))
])

dt_model.fit(X_train, y_train)


,steps,"[('preprocess', ...), ('smote', ...), ...]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('num', ...), ('cat', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [18]:
from sklearn.neighbors import KNeighborsClassifier

knn_model = ImbPipeline(steps=[
    ('preprocess', preprocessor),
    ('smote', smote),
    ('model', KNeighborsClassifier(n_neighbors=9, weights='distance'))
])

knn_model.fit(X_train, y_train)


,steps,"[('preprocess', ...), ('smote', ...), ...]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('num', ...), ('cat', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [19]:
from sklearn.ensemble import IsolationForest

iso_model = ImbPipeline(steps=[
    ('preprocess', preprocessor),
    ('model', IsolationForest(contamination=0.1, random_state=42))
])

iso_model.fit(X_train)


,steps,"[('preprocess', ...), ('model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('num', ...), ('cat', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [20]:
for name, model in {
    "RF": rf_model,
    "LR": lr_model,
    "DT": dt_model,
    "KNN": knn_model
}.items():
    print(name, "Train Acc:", model.score(X_train, y_train))
    print(name, "Test Acc :", model.score(X_test, y_test))


RF Train Acc: 0.995225102319236
RF Test Acc : 0.9482288828337875
LR Train Acc: 1.0
LR Test Acc : 1.0
DT Train Acc: 1.0
DT Test Acc : 1.0
KNN Train Acc: 1.0
KNN Test Acc : 0.40054495912806537


In [21]:
from sklearn.metrics import classification_report

print(classification_report(y_test, rf_model.predict(X_test)))


              precision    recall  f1-score   support

         0.0       0.93      1.00      0.97       545
         1.0       1.00      0.80      0.89       189

    accuracy                           0.95       734
   macro avg       0.97      0.90      0.93       734
weighted avg       0.95      0.95      0.95       734



In [22]:
import joblib

joblib.dump(rf_model, "random_forest.pkl")
joblib.dump(lr_model, "logistic_regression.pkl")
joblib.dump(dt_model, "decision_tree.pkl")
joblib.dump(knn_model, "knn.pkl")
joblib.dump(iso_model, "isolation_forest.pkl")
joblib.dump(preprocessor, "preprocessor.pkl")


['preprocessor.pkl']

In [23]:
# Get feature names after preprocessing
ohe = rf_model.named_steps['preprocess'].named_transformers_['cat']
encoded_cat_cols = ohe.get_feature_names_out(categorical_cols)

all_feature_names = list(numeric_cols) + list(encoded_cat_cols)


In [24]:
import pandas as pd

rf_importance = rf_model.named_steps['model'].feature_importances_

importance_df = pd.DataFrame({
    "Feature": all_feature_names,
    "Importance": rf_importance
}).sort_values(by="Importance", ascending=False)

print("🔝 Top Features Influencing Fraud:")
print(importance_df.head(10))


🔝 Top Features Influencing Fraud:
                                Feature  Importance
11189              fraud_type_Non-Fraud    0.168340
11190  fraud_type_Potential Claim Fraud    0.162239
7983    provider_type_Specialist Office    0.037762
3                  number_of_procedures    0.024626
12                 claim_amount_per_day    0.022860
11                        amount billed    0.021273
7985      provider_specialty_Cardiology    0.020618
13           claim_amount_per_procedure    0.020013
7990     provider_specialty_Orthopedics    0.016033
2                        procedure_code    0.012443


In [25]:
def generate_fraud_reason(input_data, importance_df):
    top_features = importance_df.head(5)["Feature"].values
    reasons = []

    for feat in top_features:
        if feat in input_data.columns:
            value = input_data[feat].values[0]
            reasons.append(f"{feat} contributed with value {value}")

    return reasons


In [26]:
sample_input = X_test.iloc[[0]]
reasons = generate_fraud_reason(sample_input, importance_df)

print("🧾 Fraud Reason Explanation:")
for r in reasons:
    print("-", r)


🧾 Fraud Reason Explanation:
- number_of_procedures contributed with value 1.0
- claim_amount_per_day contributed with value 2069401.13


In [27]:
lr_coefs = lr_model.named_steps['model'].coef_[0]

lr_importance_df = pd.DataFrame({
    "Feature": all_feature_names,
    "Coefficient": abs(lr_coefs)
}).sort_values(by="Coefficient", ascending=False)

print(lr_importance_df.head(10))


                                Feature  Coefficient
11189              fraud_type_Non-Fraud     5.109236
11190  fraud_type_Potential Claim Fraud     5.033882
7983    provider_type_Specialist Office     0.496217
7985      provider_specialty_Cardiology     0.340012
3                  number_of_procedures     0.309680
7990     provider_specialty_Orthopedics     0.307185
13           claim_amount_per_procedure     0.286198
8055            admission_type_Elective     0.178335
8059              admission_type_Urgent     0.176398
9       provider_patient_distance_miles     0.174319


In [28]:
dt_importance = dt_model.named_steps['model'].feature_importances_

dt_importance_df = pd.DataFrame({
    "Feature": all_feature_names,
    "Importance": dt_importance
}).sort_values(by="Importance", ascending=False)

print(dt_importance_df.head(10))


                      Feature  Importance
11189    fraud_type_Non-Fraud         1.0
0                 patient_age         0.0
7464    claim_amount_687909.2         0.0
7456   claim_amount_682300.29         0.0
7457   claim_amount_683691.67         0.0
7458   claim_amount_684830.45         0.0
7459   claim_amount_684919.33         0.0
7460    claim_amount_685331.6         0.0
7461   claim_amount_687016.71         0.0
7462   claim_amount_687273.82         0.0


In [29]:
distances, indices = knn_model.named_steps['model'].kneighbors(
    rf_model.named_steps['preprocess'].transform(sample_input)
)

print("🧠 KNN Explanation: Similar past cases influenced this prediction")
print("Nearest fraud case indices:", indices[0][:3])


🧠 KNN Explanation: Similar past cases influenced this prediction
Nearest fraud case indices: [3537 3422 3227]


In [30]:
anomaly_score = iso_model.named_steps['model'].decision_function(
    rf_model.named_steps['preprocess'].transform(sample_input)
)

print("⚠️ Isolation Forest Anomaly Score:", anomaly_score[0])
print("Lower score = more anomalous (fraud-like behavior)")


⚠️ Isolation Forest Anomaly Score: 0.001511276605045797
Lower score = more anomalous (fraud-like behavior)


In [31]:
import joblib

joblib.dump(rf_model, "random_forest.pkl")
joblib.dump(lr_model, "logistic_regression.pkl")
joblib.dump(dt_model, "decision_tree.pkl")
joblib.dump(knn_model, "knn.pkl")
joblib.dump(iso_model, "isolation_forest.pkl")


['isolation_forest.pkl']

In [32]:
print(type(rf_model))
print(type(lr_model))
print(type(dt_model))
print(type(knn_model))
print(type(iso_model))


<class 'imblearn.pipeline.Pipeline'>
<class 'imblearn.pipeline.Pipeline'>
<class 'imblearn.pipeline.Pipeline'>
<class 'imblearn.pipeline.Pipeline'>
<class 'imblearn.pipeline.Pipeline'>


In [33]:
rf_model.fit(X_train, y_train)


,steps,"[('preprocess', ...), ('smote', ...), ...]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('num', ...), ('cat', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [34]:
import joblib

joblib.dump(rf_model, "random_forest.pkl")
joblib.dump(lr_model, "logistic_regression.pkl")
joblib.dump(dt_model, "decision_tree.pkl")
joblib.dump(knn_model, "knn.pkl")
joblib.dump(iso_model, "isolation_forest.pkl")

print("✅ All models saved successfully")


✅ All models saved successfully


In [35]:
import os
print(os.listdir())


['.ipynb_checkpoints', 'app.py', 'app_demo.py', 'data', 'decision_tree.pkl', 'dt_feature_importance.pkl', 'fraud_detection_model.pkl', 'fraud_model.pkl', 'isolation_forest.pkl', 'knn.pkl', 'logistic_regression.pkl', 'lr_coefficients.pkl', 'model_columns.pkl', 'model_metrics.pkl', 'preprocessor.pkl', 'project1.ipynb', 'random_forest.pkl', 'requirements (1).txt', 'rf_feature_importance.pkl', 'save_models.py', 'scaler.pkl', 'training_data.csv']


In [36]:
joblib.dump(preprocessor, "preprocessor.pkl")


['preprocessor.pkl']

In [37]:
joblib.dump(X_train.columns.tolist(), "model_columns.pkl")


['model_columns.pkl']

In [38]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

model_metrics = {
    "Random Forest": {
        "accuracy": accuracy_score(y_test, rf_model.predict(X_test)),
        "precision": precision_score(y_test, rf_model.predict(X_test)),
        "recall": recall_score(y_test, rf_model.predict(X_test)),
        "f1": f1_score(y_test, rf_model.predict(X_test)),
        "roc_auc": roc_auc_score(y_test, rf_model.predict_proba(X_test)[:,1])
    },
    "Logistic Regression": {
        "accuracy": accuracy_score(y_test, lr_model.predict(X_test)),
        "precision": precision_score(y_test, lr_model.predict(X_test)),
        "recall": recall_score(y_test, lr_model.predict(X_test)),
        "f1": f1_score(y_test, lr_model.predict(X_test)),
        "roc_auc": roc_auc_score(y_test, lr_model.predict_proba(X_test)[:,1])
    },
    "Decision Tree": {
        "accuracy": accuracy_score(y_test, dt_model.predict(X_test)),
        "precision": precision_score(y_test, dt_model.predict(X_test)),
        "recall": recall_score(y_test, dt_model.predict(X_test)),
        "f1": f1_score(y_test, dt_model.predict(X_test)),
        "roc_auc": roc_auc_score(y_test, dt_model.predict_proba(X_test)[:,1])
    },
    "KNN": {
        "accuracy": accuracy_score(y_test, knn_model.predict(X_test)),
        "precision": precision_score(y_test, knn_model.predict(X_test)),
        "recall": recall_score(y_test, knn_model.predict(X_test)),
        "f1": f1_score(y_test, knn_model.predict(X_test)),
        "roc_auc": roc_auc_score(y_test, knn_model.predict_proba(X_test)[:,1])
    }
}

joblib.dump(model_metrics, "model_metrics.pkl")


['model_metrics.pkl']

In [39]:
joblib.dump(rf_model.named_steps['model'].feature_importances_, "rf_feature_importance.pkl")
joblib.dump(dt_model.named_steps['model'].feature_importances_, "dt_feature_importance.pkl")
joblib.dump(lr_model.named_steps['model'].coef_, "lr_coefficients.pkl")


['lr_coefficients.pkl']

In [40]:
df.to_csv("training_data.csv", index=False)


In [41]:
pip install joblib


Note: you may need to restart the kernel to use updated packages.


In [43]:
feature_columns = X.columns.tolist()
print("Total features:", len(feature_columns))


Total features: 39


In [44]:
import joblib

joblib.dump(rf_model, "random_forest.pkl")
joblib.dump(lr_model, "logistic_regression.pkl")
joblib.dump(dt_model, "decision_tree.pkl")
joblib.dump(knn_model, "knn.pkl")
joblib.dump(iso_model, "isolation_forest.pkl")

joblib.dump(preprocessor, "preprocessor.pkl")
joblib.dump(feature_columns, "model_columns.pkl")

print("✅ All models saved correctly")


✅ All models saved correctly
